In [1]:
pip install wikipedia-api openai faiss-cpu sentence-transformers

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.0 MB/s eta 0:00:00
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl si

**Console bot**

In [3]:
pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.49
    Uninstalling langchain-core-0.3.49:
      Successfully uninstalled langchain-core-0.3.49
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [4]:
import requests
import faiss
import numpy as np
import wikipediaapi
from sentence_transformers import SentenceTransformer
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import getpass

# ✅ Get OpenRouter API key
api_key = getpass.getpass("Enter your OpenRouter API key: ")

# ✅ OpenRouter API URL
API_URL = "https://openrouter.ai/api/v1/chat/completions"

# ✅ Wikipedia API
wiki_wiki = wikipediaapi.Wikipedia(user_agent="MyWikipediaChatbot/1.0", language="en")

# ✅ Embedding Model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# ✅ FAISS Index for storing Wikipedia summaries
embedding_dim = 384  # Dimension of the MiniLM model
index = faiss.IndexFlatL2(embedding_dim)
stored_texts = []  # Store Wikipedia text

# ✅ Fetch Wikipedia summary
def fetch_wikipedia_summary(topic):
    page = wiki_wiki.page(topic)
    if page.exists():
        return page.summary
    return None

# ✅ Convert text to embeddings
def embed_text(text):
    return embedder.encode([text])[0]

# ✅ Add Wikipedia text to FAISS
def add_to_index(text):
    vector = embed_text(text)
    index.add(np.array([vector], dtype=np.float32))
    stored_texts.append(text)

# ✅ Retrieve relevant text using FAISS
def retrieve_similar_text(query, top_k=1):
    if len(stored_texts) == 0:
        return ["No relevant Wikipedia data found."]

    query_vector = embed_text(query).reshape(1, -1)
    distances, indices = index.search(query_vector, top_k)
    return [stored_texts[i] for i in indices[0] if i < len(stored_texts)]

# ✅ Set up OpenRouter LLM in LangChain
llm = ChatOpenAI(
    model="mistralai/mistral-small",  # Use OpenRouter-supported model
    openai_api_key=api_key,
    openai_api_base="https://openrouter.ai/api/v1"
)

# ✅ Prompt Template
prompt_template = PromptTemplate(
    input_variables=["context", "query"],
    template="Context: {context}\nQuestion: {query}\nAnswer:"
)

# ✅ LangChain LLM Chain
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

# ✅ Generate response using OpenRouter LLM
def generate_response(query):
    context = retrieve_similar_text(query)
    context_str = "\n".join(context) if context else "No relevant Wikipedia data found."

    response = llm_chain.invoke({"context": context_str, "query": query})
    return response

# ✅ Chatbot loop
def start_chat():
    print("📚 Wikipedia Chatbot (RAG) - Type 'exit' to quit.")

    while True:
        query = input("\nYou: ")
        if query.lower() == "exit":
            print("Goodbye! 👋")
            break

        if query not in stored_texts:
            summary = fetch_wikipedia_summary(query)
            if summary:
                add_to_index(summary)

        response = generate_response(query)
        print(f"🤖 Chatbot: {response}")

# ✅ Run chatbot
start_chat()


Enter your OpenRouter API key: ··········


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-4-0eacceea2014>:55: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
<ipython-input-4-0eacceea2014>:68: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt_template)


📚 Wikipedia Chatbot (RAG) - Type 'exit' to quit.

You: When was MS Dhoni Born?
🤖 Chatbot: {'context': 'No relevant Wikipedia data found.', 'query': 'When was MS Dhoni Born?', 'text': 'MS Dhoni, full name Mahendra Singh Dhoni, was born on July 7, 1981.'}

You: Explain about Mahendra Singh Dhoni
🤖 Chatbot: {'context': 'No relevant Wikipedia data found.', 'query': 'Explain about Mahendra Singh Dhoni', 'text': "Mahendra Singh Dhoni, often referred to as MS Dhoni, is a former Indian international cricketer who is widely regarded as one of the greatest finishers and captains in the history of the game. Here are some key aspects of his career:\n\n1. **Early Life and Career:**\n   - Born on July 7, 1981, in Ranchi, Jharkhand, Dhoni started his career as a ticket collector for the Indian Railways.\n   - He began playing cricket at a young age and was selected for the Indian team in 2004.\n\n2. **Cricket Career:**\n   - **Batsman:** Dhoni is known for his aggressive batting style, particularly h

KeyboardInterrupt: Interrupted by user

**Gradio bot**

In [ ]:
import gradio as gr
import requests
import faiss
import numpy as np
import wikipediaapi
from sentence_transformers import SentenceTransformer
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory   # Updated Memory
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# ✅ OpenRouter API Configuration
API_KEY = getpass.getpass("Enter your OpenRouter API key: ")
BASE_URL = "https://openrouter.ai/api/v1"

# ✅ Wikipedia API
wiki = wikipediaapi.Wikipedia(user_agent="MyWikipediaChatbot/1.0", language="en")

# ✅ Load Sentence Transformer for Embeddings
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embedding_dim = 384

# ✅ FAISS Vector Store
index = faiss.IndexFlatL2(embedding_dim)
stored_texts = []

# ✅ LLM Model (OpenRouter)
llm = ChatOpenAI(
    model="mistralai/mistral-small",
    openai_api_key=API_KEY,
    openai_api_base=BASE_URL
)

# ✅ Updated Memory (Fixing Deprecation Warning)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# ✅ Function to Fetch Wikipedia Summary
def fetch_wikipedia_summary(topic):
    page = wiki.page(topic)
    return page.summary if page.exists() else None

# ✅ Convert Text to Embeddings
def embed_text(text):
    return embedder.encode([text])[0]

# ✅ Add Wikipedia Text to FAISS
def add_to_index(text):
    vector = embed_text(text)
    index.add(np.array([vector], dtype=np.float32))
    stored_texts.append(text)

# ✅ Retrieve Similar Text
def retrieve_similar_text(query, top_k=1):
    if len(stored_texts) == 0:
        return ["No relevant Wikipedia data found."]

    query_vector = embed_text(query).reshape(1, -1)
    distances, indices = index.search(query_vector, top_k)
    return [stored_texts[i] for i in indices[0] if i < len(stored_texts)]

# ✅ Create Prompt Template
prompt = PromptTemplate(
    input_variables=["input_text"],  # Use a single variable
    template="""
    You are an AI assistant answering questions using Wikipedia.
    {input_text}
    Answer:
    """
)


# ✅ Create LLM Chain
llm_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

# ✅ Generate Response
def generate_response(query):
    if query not in stored_texts:
        summary = fetch_wikipedia_summary(query)
        if summary:
            add_to_index(summary)

    context = retrieve_similar_text(query)
    context_str = "\n".join(context) if context else "No relevant Wikipedia data found."

    input_text = f"Context: {context_str}\nQuery: {query}"

    response = llm_chain.run(input_text)  # Now using a single input variable
    return response


# ✅ Create Gradio Interface (Fixed `debug` Issue)
def chatbot_interface(query):
    return generate_response(query)

gr.Interface(
    fn=chatbot_interface,
    inputs="text",
    outputs="text",
    title="📚 Wikipedia Chatbot (RAG)",
    description="Ask me anything! I use Wikipedia to find relevant info and generate responses.",
    theme="default"  # Fixed theme issue
).launch(debug=True)  # Automatically sets `share=True` in Colab


<ipython-input-7-22b00c10c14b>:37: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://116e569b469196f8bc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<ipython-input-7-22b00c10c14b>:89: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_chain.run(input_text)  # Now using a single input variable


In [6]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.5 MB/s eta 0:00:00
